In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import squarify
import json
import os
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [ ]:
all = pd.read_csv('crawl_weather.csv')
all_sort = all.sort_values(by='url').drop(['url.1', '...'],axis=1).reset_index(drop=True)

In [ ]:
all_sort.to_csv('all_sort.csv',index=0)
all_sort = pd.read_csv('all_sort.csv')

In [ ]:
cities = all_sort['url']
city = []
for i in range(0,len(cities)):
    city.append(cities[i][24:-12])
city.sort()

In [ ]:
all_sort.insert(0,'city',city)
all_sort.drop(columns='url',inplace=True)
all_sort.to_csv('all_sort2.csv',index=0)
all_sort = pd.read_csv('all_sort2.csv')

In [ ]:
counts = all_sort['city'].value_counts(sort=False,dropna=False)
counts = counts.sort_index()
j = 0
a = pd.DataFrame
city_map = {}
for i in range(0,len(counts)):
    a = all_sort.iloc[j:j + counts[i],:]
    ti = a['title'][j + counts[i] - 1].split('|')
    city_map[ti[0][:-6]] = counts.index[i]
    a.to_csv('E:\\vscode files\python\csv\\' + counts.index[i],index=0) 
    j += counts[i]

In [ ]:
# 建立索引，方便查找城市名和对应缩写
city_map['鼓楼'] = city_map.pop('徐州鼓楼历史天气查询_徐州鼓楼历史天气预报_徐州鼓楼历史天气记录_徐州鼓楼历史')
city_map['简阳'] = city_map.pop('简阳历史天气查询_简阳历史天气预报_简阳历史天气记录_简阳历史')
city_map['桥东'] = city_map.pop('桥东历史天气查询_桥东历史天气预报_桥东历史天气记录_桥东历史')
city_map['市中'] = city_map.pop('枣庄市中历史天气查询_枣庄市中历史天气预报_枣庄市中历史天气记录_枣庄市中历史')
city_map['杨陵'] = city_map.pop('杨陵历史天气查询_杨陵历史天气预报_杨陵历史天气记录_杨陵历史')
json.dump(city_map, open('./city_map', "w"))

In [ ]:
def data_fmt(city) :
    try :
        df = pd.read_csv('./csv/%s' %city)
    except :
        print('文件不存在')
        return 0
    if os.path.exists('./done_csv/done_%s.csv' %city) :
        return 0
    
    # 去除nan
    df.dropna(inplace=True)
    
    # 插入表头
    header = df['head'].values[0].split(" ")
    header.append("风力")
    header.insert(1,'日')
    header.insert(1,'月')
    header.insert(1,'年')



    # 处理日期信息
    date = []
    for i in df.index :
        date.append(df['date'][i])
    for i in range(0,len(date)) :
        date[i] = date[i].replace(' 星期一','').replace(' 星期二','').replace(' 星期三','').replace(' 星期四','').replace(' 星期五','').replace(' 星期六','').replace(' 星期日','').replace(' 周一','').replace(' 周二','').replace(' 周三','').replace(' 周四','').replace(' 周五','').replace(' 周六','').replace(' 周日','')
    date = " ".join(date).split(" ")

    # 细分年月日信息
    year = []
    month = []
    day = []
    for i in range(0,len(date)) :
        year.append(date[i][:4])
        month.append(date[i][5:-3])
        day.append(date[i][-2:])
    
    # 处理数据格式
    contents = []
    for i in df.index :
        contents.append(df['content'][i])
    contents = " ".join(contents).split(" ")
    
    # 填补空缺值
    wrong_index = []
    for i in range(0,len(contents)) :
        if contents[i] == '暂无实况':
            wrong_index.append(i)
        if contents[i] == '' :
            contents[i] = '暂无实况'
            if '℃' in contents[i - 3] : 
                wrong_index.append(i)

    for i in range(0,len(wrong_index)) :
        contents.insert(wrong_index[i]+1, "暂无实况")

    # 分配数据
    high_temp = []
    low_temp = []
    weather = []
    windType = []
    windPower = []
    for i in range(0,len(contents)) :
        if i%5 == 0 :
            high_temp.append(contents[i][:-1])
        elif i%5 == 1 :
            low_temp.append(contents[i][:-1])
        elif i%5 == 2 :
            weather.append(contents[i])
        elif i%5 == 3 :
            windType.append(contents[i])
        elif i%5 == 4 :
            windPower.append(contents[i])
        else :
            print("conut again")
    
    # 创建表格
    df_all = pd.DataFrame(columns=header,index=(i for i in range(0,len(date))))
    for i in range(0,len(date)) :
        df_all[header[0]][i] = date[i]
        df_all[header[1]][i] = year[i]
        df_all[header[2]][i] = month[i]
        df_all[header[3]][i] = day[i]
        df_all[header[4]][i] = int(high_temp[i])
        df_all[header[5]][i] = int(low_temp[i])
        df_all[header[6]][i] = weather[i]
        df_all[header[7]][i] = windType[i]
        df_all[header[8]][i] = windPower[i]

    # 气温纠正
    tem_index = df_all[df_all['最低气温']>df_all['最高气温']].index
    for i in range(0,len(tem_index)) :
        temp1 = df_all.loc[tem_index[i]]['最高气温']
        temp2 = df_all.loc[tem_index[i]]['最低气温']
        df_all['最高气温'][tem_index[i]] = temp2
        df_all['最低气温'][tem_index[i]] = temp1
    # 保存
    df_all.to_csv('./done_csv/done_%s.csv' %city,index=0)
    return df_all

In [ ]:
def fmt_to_pic(city) :
    try :
        df_all = pd.read_csv('./done_csv/done_%s.csv' %city)
    except :
        print('文件不存在')
        return 0

    # 以年为单位划分表
    year_list = df_all['年'].unique()
    df_list = []
    
    for i in range(0,len(year_list)):
        df_list.append(df_all[df_all['年'].isin([year_list[i]])])


    # 最高低温度图
    for i in range(0,len(df_list)):
        fig = plt.figure(dpi=300,figsize=(60,8))
        plt.xlabel('日期: 年-月-日')
        plt.ylabel('温度: ℃')
        plt.title('%s-%s年每日温度-折线图' %(city, df_list[i]['年'].values[0]))
        plt.plot(df_list[i]['日期'],df_list[i]['最高气温'],'r-', label='最高气温')
        plt.plot(df_list[i]['日期'],df_list[i]['最低气温'], 'b-', label='最低气温')
        high_mean = np.mean(df_list[i]['最高气温'])
        low_mean = np.mean(df_list[i]['最低气温'])
        plt.axhline(y=high_mean,ls='--',color='r',label='平均最高气温')
        plt.axhline(y=low_mean,ls='--',color='b',label='平均最低气温')
        fig.autofmt_xdate(rotation=45)
        plt.legend(loc='upper right')
        plt.savefig("./picture/%s-%s-温度图.png" %(city, df_list[i]['年'].values[0]))

    # 天气占比树地图
    for i in range(0,len(df_list)):
        plt.figure(dpi=500,figsize=(24,16))
        label_sq = df_list[i]['天气'].value_counts().index
        sizes = df_list[i]['天气'].value_counts().values
        plot = squarify.plot(sizes=sizes, norm_x=300, norm_y=300, label=label_sq, value=sizes, edgecolor='white', alpha=0.6, linewidth=3)
        plot.set_title('%s-%s年天气占比状况-树地图' %(city, df_list[i]['年'].values[0]))
        plt.rc('font', size=8)
        plt.axis('off')
        plt.tick_params(top = 'off', right = 'off')
        plt.savefig("./picture/%s-%s-天气图.png" %(city, df_list[i]['年'].values[0]))

    # 主要风向占比饼状图
    for i in range(0,len(df_list)):
        plt.figure(dpi=100,figsize=(20,20))
        length = len(df_list[i]['风向'].value_counts())
        label = df_list[i]['风向'].value_counts().index
        values = df_list[i]['风向'].value_counts().values
        if length >= 10 :
            num = 10
            label = label[:num]
            values = values[:num]
            explode = [d for d in np.arange(0.1,0,-0.01)]
        elif length >= 5:
            num = 5
            label = label[:num]
            values = values[:num]
            explode = [0.1,0.07,0.04,0.01,0]
        else :
            num = 3
            label = label[:num]
            values = values[:num]
            explode = [0.1,0.07,0.04]
        plt.pie(values,labels=label,explode=explode,autopct='%1.1f%%', wedgeprops = {'linewidth': 1.5, 'edgecolor':'green'}, textprops={'fontsize':12, 'color':'k'})
        plt.title('%s-%s年风向占比状况-饼状图' %(city, df_list[i]['年'].values[0]))
        plt.savefig("./picture/%s-%s-风向比例图.png" %(city, df_list[i]['年'].values[0]))
    
    return 1